In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [2]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_midcpnifty(f_today):

    days_to_thursday = (0 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [3]:
# bnf_pandas = pd.read_csv("../data/bnf_1hr_tv (9).csv")
bnf_pandas = pd.read_csv("../data/fin.csv")
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2023]
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')

In [4]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [5]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [6]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(bnf, '5m', pd.Timedelta(minutes=0))

# bnf_1hr = ohlc_resampled
# bnf_final = bnf_1hr.to_pandas()
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final
# bnf_1hr

In [8]:
def generate_signals(df, signal_ma, candles_in_num):
    df["c"] = pd.to_numeric(df["c"], errors="coerce")
    df["Signal MA"] = df["c"].rolling(window=signal_ma).mean()
    # df["Trailing MA"] = df["c"].rolling(window=trailing_ma).mean()

    df["Sell Signal"] = 0

    # Generate signals using boolean masking
    sell_signal_mask = df["Signal MA"] <= df["Signal MA"].shift(1)

    for i in range(1, candles_in_num):
        sell_signal_mask &= df["Signal MA"].shift(i) <= df["Signal MA"].shift(i + 1)

    df.loc[sell_signal_mask, "Sell Signal"] = 1

    return df

In [9]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].tail())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

869531   2024-06-07 15:25:00
869532   2024-06-07 15:26:00
869533   2024-06-07 15:27:00
869534   2024-06-07 15:28:00
869535   2024-06-07 15:29:00
Name: datetime, dtype: datetime64[ns]


In [38]:
# GLOBAL VARIABLES

# INSTRUMENT = "BANKNIFTY"
# INDEX = "bnf"

# INSTRUMENT = "MIDCPNIFTY"
# INDEX = "midcpnifty"

INSTRUMENT = "FINNIFTY"
INDEX = "finnifty"

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"

PORTFOLIO_VALUE = 10_00_000
INDEX_LEV = 20

In [11]:
async def trade_expiry(df, signal_ma, trailing_ma, time_of_day):

    start_date = dt.date(2017, 1, 1)
    end_date = dt.date(2024, 7, 31)

    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()

    while current_date <= end_date:
        if current_date in trading_days_set:

            starting_time = dt.time(9, 15)
            # time_of_day = dt.time(13, 0)
            ending_time = dt.time(15, 30)

            search_datetime = dt.datetime.combine(current_date, time_of_day)

            spot_open = df.loc[df["datetime"] >= search_datetime, "open"].iloc[0]
            # print(df.loc[df['datetime'] >= search_datetime, 'datetime'].iloc[0])
            # print(spot_open)

            spot_atm = int(round(spot_open / 25) * 25)
            # print(spot_atm)

            nearest_expiry = await get_expiry_midcpnifty(current_date)
            # print(nearest_expiry)

            if current_date == nearest_expiry:
                # print(f"Current Date : {current_date}")

                ce_df = await fetch_data(
                    index=INDEX,
                    start_date=current_date,
                    start_time=starting_time,
                    end_date=current_date,
                    end_time=ending_time,
                    strike=spot_atm,
                    asset_class="C",
                    expiry=nearest_expiry,
                )
                if not isinstance(ce_df, str) and (ce_df is not None):
                    # print("IF CONDITION TRIGGERED -> CE Option")
                    # ce_df = ce_df.rename(
                    #     {"open": "o", "high": "h", "low": "l", "close": "c"}
                    # )
                    ce_df = ce_df.select(["datetime", "o", "h", "l", "c"])
                    ce_df = resample(ce_df, "5m")
                    ce_df_pandas = ce_df.to_pandas()
                    ce_df = generate_signals1(ce_df_pandas, signal_ma, trailing_ma, 1)
                    # ce_df = generate_signals2(ce_df_pandas)
                    # print(ce_df.tail())
                else:
                    # print("ELSE CONDITION TRIGGERED -> CE Option")
                    # print(ce_df)
                    current_date += dt.timedelta(days=1)
                    continue

                pe_df = await fetch_data(
                    index=INDEX,
                    start_date=current_date,
                    start_time=starting_time,
                    end_date=current_date,
                    end_time=ending_time,
                    strike=spot_atm,
                    asset_class="P",
                    expiry=nearest_expiry,
                )
                if not isinstance(pe_df, str) and (pe_df is not None):
                    # print("IF CONDITION TRIGGERED -> PE Option")
                    # pe_df = pe_df.rename(
                    #     {"open": "o", "high": "h", "low": "l", "close": "c"}
                    # )
                    pe_df = pe_df.select(["datetime", "o", "h", "l", "c"])
                    pe_df = resample(pe_df, "5m")
                    pe_df_pandas = pe_df.to_pandas()
                    pe_df = generate_signals1(pe_df_pandas, signal_ma, trailing_ma, 1)
                    # pe_df = generate_signals2(pe_df_pandas)
                    # print(pe_df.tail())
                else:
                    # print("ELSE CONDITION TRIGGERED -> PE Option")
                    # print(pe_df)
                    current_date += dt.timedelta(days=1)
                    continue

                # print(
                #     ce_df_pandas.to_string(), "\n", pe_df_pandas.to_string()
                # )

                # print('WORKING ON CE TRADES')
                ce_trades = await trade_mtrend_option_buying(
                    df=ce_df_pandas,
                    index=INDEX,
                    strike=spot_atm,
                    asset_class="C",
                    expiry=nearest_expiry,
                    search_datetime=search_datetime,
                    # starting_index=starting_index,
                )
                # print('WORKING ON PE TRADES')
                pe_trades = await trade_mtrend_option_buying(
                    df=pe_df_pandas,
                    index=INDEX,
                    strike=spot_atm,
                    asset_class="P",
                    expiry=nearest_expiry,
                    search_datetime=search_datetime,
                    # starting_index=starting_index,
                )
                combined_trades = pd.concat(
                    [combined_trades, ce_trades], ignore_index=True
                )
                combined_trades = pd.concat(
                    [combined_trades, pe_trades], ignore_index=True
                )
                # print(combined_trades)

        current_date += dt.timedelta(days=1)

    total_trades = pd.concat([total_trades, combined_trades], ignore_index=True)

    return total_trades


# tb = await trade_expiry(data)
# tb

In [12]:
data = data.reset_index(drop=True)
data.set_index("datetime", inplace=True)
# data.tail()

In [13]:
async def trade_option_selling(df, index, strike, asset_class, expiry, signal_ma):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    eod_exit_flag = False
    remark = ""
    entry_time = None
    no_more_trades = False

    df['datetime'] = pd.to_datetime(df['datetime'])

    for i in range(0, len(df)):
        # print(df.iloc[i]['datetime'])
        if not no_more_trades:
            
            points = 0
            current_candle_open = df.iloc[i]["o"]
            current_candle_high = df.iloc[i]["h"]
            current_candle_low = df.iloc[i]["l"]
            current_candle_close = df.iloc[i]["c"]
            subset_df = df[max(0, i-signal_ma):i+1]

            if not in_trade:

                    
                if df.iloc[i]["Sell Signal"] == 1:
                    if not already_signal_exists:
                        # Fresh Sell Signal
    
                        # print(df[df['h'] == max(subset_df['h'])].to_string())
                        # print("Fresh Sell Signal")
                        signal_entry_price = current_candle_low
                        # signal_initial_sl = current_candle_low * 1.5
                        signal_initial_sl = max(subset_df['h'])
                        signal_creation_time = df.iloc[i]["datetime"]
                        already_signal_exists = True
                        # print(signal_creation_time)
    
                    else:
                        if current_candle_low > signal_entry_price:
                            # Better Candle
                            # print(df[df['h'] == max(subset_df['h'])].to_string())
                            # print("Better Candle")
                            signal_entry_price = current_candle_low
                            # signal_initial_sl = current_candle_low * 1.5
                            signal_initial_sl = max(subset_df['h'])
                            signal_creation_time = df.iloc[i]["datetime"]
                            # print(signal_entry_price, signal_initial_sl)
                            # print(signal_creation_time)
    
                        elif current_candle_low <= signal_entry_price:
    
                            if (
                                current_candle_open < signal_entry_price
                                and df.iloc[i]["datetime"].date()
                                > df.iloc[i - 1]["datetime"].date()
                            ):
                                # Gap Down Condition, Skip Entry
    
                                # print("Gap Down Open, Skip Trade")
                                already_signal_exists = False
                                signal_entry_price = 100000
                                signal_initial_sl = 0
    
                            else:
                                # Entry Triggered
        
                                # print("Entry Triggered")
                                in_trade = True
                                entry_time = df.iloc[i]["datetime"]
                                entry_price = signal_entry_price
                                # print(entry_price, signal_initial_sl)
                                points = 0
    
                else:
                    # Signal Does Not Exist In this Candle
                    if (df.iloc[i - 1]["Sell Signal"] == 1):
                        # print(df[df['h'] == max(subset_df['h'])].to_string())
                        # Condition to Enter the trade even if current candle is not Signal Candle
                        signal_entry_price = df.iloc[i - 1]["l"]
                        # signal_initial_sl = df.iloc[i - 1]["l"] * 1.5
                        signal_initial_sl = max(subset_df['h'])
                        signal_creation_time = df.iloc[i - 1]["datetime"]
                        already_signal_exists = True
                        # print(signal_entry_price, signal_initial_sl)
                        # print(signal_creation_time)
        
                        if current_candle_low <= signal_entry_price and df.iloc[i]['datetime'] != df.iloc[0]['datetime']:
                            # Entry Triggered
    
                            # print("Entry Triggered")
                            in_trade = True
                            entry_time = df.iloc[i]["datetime"]
                            entry_price = signal_entry_price
                            points = 0
                            # print(entry_price, signal_initial_sl)
                        else:
                            # Discard Existing Signal
                            
                            # print('Signal Discarded')
                            already_signal_exists = False
                            signal_entry_price = 100000
                            signal_initial_sl = 0
    
                # if entry_time:
                #     if entry_time.date() == df.iloc[0].date() and entry_time.time() == dt.time(9, 15):
                #         points = 0
                #         in_trade = False
                #         already_signal_exists = False
                #         remark = ""
                #         is_trailing_active = False
                #         # no_more_trades = False
                #         continue
                    
            if in_trade:
                # print(df.iloc[i]['datetime'])
                trade_entry_price = signal_entry_price
                trade_initial_sl = signal_initial_sl
                trade_target = 10

                # print(trade_entry_price, trade_initial_sl)

                if trade_entry_price < 10:
                    in_trade = False
                    points = 0
                    # continue
                else:
                    if current_candle_open > trade_initial_sl:
                        if (
                            df.iloc[i]["datetime"].date() == entry_time.date()
                            and df.iloc[i]["datetime"].time() == entry_time.time()
                        ):
                            if current_candle_close >= trade_initial_sl:
                                # print('Initial SL Hit')
                                in_trade = False
                                points = -1 * (trade_initial_sl - trade_entry_price)
                                exit_price = trade_initial_sl
                                exit_time = df.iloc[i]["datetime"]
                                remark = "Initial SL hit"
                                # print(points)
        
                        else:
                            # Gap Open Outside ISL
        
                            # print(df.iloc[i])
                            # print('Gap Open Outside ISL')
                            in_trade = False
                            points = trade_entry_price - current_candle_close
                            exit_price = current_candle_close
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Gap Outside ISL"
                            # print(points)
        
                    elif current_candle_high >= trade_initial_sl:
                        # Initial SL Hit
        
                        # print('Initial SL Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (trade_initial_sl - trade_entry_price)
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL Hit"
                        # print(points)
        
                    elif current_candle_low <= trade_target:
                        # Target Hit
        
                        # print('Target Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (trade_target - trade_entry_price)
                        exit_price = trade_target
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Target Hit"
                        no_more_trades = True
                        # print(points)
        
                    elif (df.iloc[i]['datetime'].time() >= dt.time(15, 20)) and (df.iloc[i]['datetime'].date() == expiry):
                        # Expiry Exit
        
                        # print('Expiry Exit Condition Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (current_candle_close - trade_entry_price)
                        exit_price = current_candle_close
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Expiry Exit"
                        no_more_trades = True
                        # print(points)
                        
                    if points:
                        # print(entry_time)
                        if (entry_time.date() == expiry) and (entry_time.time() >= dt.time(14, 0)):
                            # print('IF CONDITION Triggered in if points:')
                            points = 0
                            in_trade = False
                            already_signal_exists = False
                            remark = ""
                            is_trailing_active = False
                            entry_time = None
                            # no_more_trades = False
                            # continue
                        
                        else:
                            # print('ELSE CONDITION Triggered in if points:')
                            # qty = int(round(portfolio_value * 5 / entry_price / 15)) * 15
                            qty = int(round(PORTFOLIO_VALUE * INDEX_LEV / strike / 15)) * 15
                            slippage = 0.01 * (entry_price + exit_price)
                            # slippage = 10
                            final_points = points - slippage
                            # final_points = points
                            trade = {
                                "Index": index,
                                "Strike": strike,
                                "Option Type": asset_class,
                                "Expiry": expiry,
                                "Signal Generated At": signal_creation_time,
                                "Trade Type": "SELL",
                                "Entry Date": entry_time.date(),
                                "Entry Time": entry_time.time(),
                                "Entry Price": entry_price,
                                "Initial SL": trade_initial_sl,
                                "Target": trade_target,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Points Captured": points,
                                "Slippages": slippage,
                                "After Costs": final_points,
                                "PnL": final_points * qty,
                                "Remarks": remark,
                                "Qty": qty,
                                # "Leverage": "5x",
                                "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                                "Trade Year": entry_time.year,
                                "Trade Month": entry_time.month,
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # print("Appended : \n", trade)
                            points = 0
                            in_trade = False
                            already_signal_exists = False
                            remark = ""
                            is_trailing_active = False
                            entry_time = None
                            # no_more_trades = False
                    
        # if no_more_trades:
    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [14]:
# async def convert_to_weekly(df):

#     before_df = df[df.index.date < dt.date(2023, 8, 31)]
#     after_df = df[df.index.date >= dt.date(2023, 8, 31)]

#     before_df_resampled = before_df.resample(
#         "W-THU", label="right", closed="right"
#     ).agg(
#         {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
#     )

#     after_df_resampled = after_df.resample(
#         "W-WED", label="right", closed="right"
#     ).agg(
#         {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
#     )

#     weekly_data = pd.concat([before_df_resampled, after_df_resampled])

#     # Reset index if you want to make the DateTime a column again
#     weekly_data = weekly_data.reset_index()

#     return weekly_data


# weekly_data = await convert_to_weekly(data)
# weekly_data.tail(100)

async def convert_to_weekly(df):

    # before_df = df[df.index.date < dt.date(2023, 8, 31)]
    # after_df = df[df.index.date >= dt.date(2023, 8, 31)]

    before_df_resampled = df.resample(
        "W-TUE", label="right", closed="right"
    ).agg(
        {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
    )

    # after_df_resampled = after_df.resample("W-WED", label="right", closed="right").agg(
    #     {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
    # )

    weekly_data = pd.DataFrame(before_df_resampled)

    # Reset index if you want to make the DateTime a column again
    weekly_data = weekly_data.reset_index()

    return weekly_data


weekly_data = await convert_to_weekly(data)
# weekly_data.tail(100)

In [15]:
weekly_data["datetime"] = pd.to_datetime(weekly_data["datetime"])
weekly_data["Week Start"] = weekly_data["datetime"].dt.date - dt.timedelta(days=6)
weekly_data["Expiry"] = weekly_data["datetime"].dt.date

for i in range(0, len(weekly_data)):
    current_expiry = weekly_data["Expiry"].iloc[i]
    if nse.valid_days(start_date=current_expiry, end_date=current_expiry).empty:
        weekly_data["Expiry"].iloc[i] = current_expiry - dt.timedelta(days=1)
print(weekly_data.to_string())
# await trade(weekly_data)

      datetime       open       high        low      close  volume  Week Start      Expiry
0   2017-01-03  7442.4000  7442.4000  7250.0000  7317.0500       0  2016-12-28  2017-01-03
1   2017-01-10  7325.1000  7416.6000  7257.2000  7409.9000       0  2017-01-04  2017-01-10
2   2017-01-17  7444.6000  7679.6500  7429.0000  7614.3500       0  2017-01-11  2017-01-17
3   2017-01-24  7627.6500  7702.7500  7480.7000  7634.2000       0  2017-01-18  2017-01-24
4   2017-01-31  7658.6000  8006.7500  7658.6000  7906.7000       0  2017-01-25  2017-01-31
5   2017-02-07  7920.9500  8244.9500  7917.6500  8196.8000       0  2017-02-01  2017-02-07
6   2017-02-14  8209.8500  8247.3500  8092.8000  8163.2000       0  2017-02-08  2017-02-14
7   2017-02-21  8171.1500  8460.8000  8116.6000  8399.1000       0  2017-02-15  2017-02-21
8   2017-02-28  8429.8500  8435.5000  8272.8500  8292.7000       0  2017-02-22  2017-02-28
9   2017-03-07  8317.3500  8428.4500  8221.2000  8280.9000       0  2017-03-01  2017-03-07

/tmp/ipykernel_4453/2745470051.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  weekly_data["Expiry"].iloc[i] = current_expiry - dt.timedelta(days=1)
/tmp/ipykernel_4453/2745470051.py:8: SettingWithCopyWarning: 
A value is trying to be se

In [45]:
async def trade(df, signal_ma, no_of_candles, tf):

    combined_trades_ce = pd.DataFrame()
    combined_trades_pe = pd.DataFrame()
    total_trades = pd.DataFrame()

    # for i in range(105, len(df)-275):
    for i in range(314, len(df)):
        current_date = df["Week Start"].iloc[i]
        current_expiry = df["Expiry"].iloc[i]
        # print(i, current_date)
        spot_open = df["open"].iloc[i]
        spot_atm = int(round(spot_open / 100) * 100)
        # print(spot_open, spot_atm)

        ce_df = await fetch_data(
            index=INDEX,
            start_date=current_date,
            end_date=current_expiry,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 30),
            expiry=current_expiry,
            strike=spot_atm,
            asset_class="C",
        )

        if not isinstance(ce_df, str) and ce_df is not None:
            ce_df = ce_df.select(["datetime", "o", "h", "l", "c"])
            ce_df = resample(ce_df, tf)
            ce_df_pandas = ce_df.to_pandas()
            ce_df = generate_signals(ce_df_pandas, signal_ma, no_of_candles)
            # print(spot_atm, 'CE DF :\n')
            # print(ce_df.to_string())

        else:
            continue

        pe_df = await fetch_data(
            index=INDEX,
            start_date=current_date,
            end_date=current_expiry,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 30),
            expiry=current_expiry,
            strike=spot_atm,
            asset_class="P",
        )

        if not isinstance(pe_df, str) and pe_df is not None:
            pe_df = pe_df.select(["datetime", "o", "h", "l", "c"])
            pe_df = resample(pe_df, tf)
            pe_df_pandas = pe_df.to_pandas()
            pe_df = generate_signals(pe_df_pandas, signal_ma, no_of_candles)
            # print(spot_atm, 'PE DF :\n')
            # print(pe_df.to_string())

        ce_trades = await trade_option_selling(
            df=ce_df,
            index=INDEX,
            strike=spot_atm,
            asset_class="C",
            expiry=current_expiry,
            signal_ma=signal_ma,
        )

        combined_trades_ce = pd.concat([combined_trades_ce, ce_trades], ignore_index=True)
        # print(combined_trades_ce.to_string())
        
        pe_trades = await trade_option_selling(
            df=pe_df,
            index=INDEX,
            strike=spot_atm,
            asset_class="P",
            expiry=current_expiry,
            signal_ma=signal_ma,
        )

        combined_trades_pe = pd.concat([combined_trades_pe, pe_trades], ignore_index=True)
        # print(combined_trades_pe.to_string())

    total_trades = pd.concat([total_trades, combined_trades_ce, combined_trades_pe], ignore_index=True)
    total_trades = total_trades.sort_values(by="Signal Generated At", ignore_index=True)
    return total_trades 

In [40]:
# weekly_data

In [41]:
def generate_stats(tb_expiry, signal_ma, no_of_candles, tf):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f'{signal_ma} , {no_of_candles}, {tf}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [50]:
x = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
y = [1, 2, 3, 4, 5]
z = ["3m", "5m", "10m", "15m", "30m"]

stats_dictionary = {}
for i in x:
    for j in y:
        for k in z:
            print(f"Signal MA : {i} , No. of Candles : {j} , TimeFrame : {k}")
            tb = await trade(weekly_data, i, j, k)
            # print(len(tb))
            if len(tb) > 0:
                tb_ce = tb[tb['Option Type'] == 'C']
                tb_pe = tb[tb['Option Type'] == 'P']
                stats = generate_stats(tb, i, j, k)
                stats_ce = generate_stats(tb_ce, i, j, k)
                stats_pe = generate_stats(tb_pe, i, j, k)

                for overall_roi_dd_ratio, stats_df in stats.items():
                    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
                        print("Overall Combined")
                        print(stats_df.to_string())
                        stats_dictionary[overall_roi_dd_ratio] = stats_df

                for overall_roi_dd_ratio, stats_df in stats_ce.items():
                    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
                        print("Only CE")
                        print(stats_df.to_string())
                        stats_dictionary[overall_roi_dd_ratio] = stats_df

                for overall_roi_dd_ratio, stats_df in stats_pe.items():
                    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
                        print("Only PE")
                        print(stats_df.to_string())
                        stats_dictionary[overall_roi_dd_ratio] = stats_df

Signal MA : 5 , No. of Candles : 1 , TimeFrame : 3m
Overall Combined
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio  Variation
2017       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  5 , 1, 3m
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  5 , 1, 3m
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  5 , 1, 3m
2020       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  5 , 1, 3m
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  5 , 1, 3m
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  5 , 1, 3m
2023       3.0848          474  19.4093               12.8691             -3.0913    -133.3357    

/tmp/ipykernel_4453/1769088592.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Overall Combined
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio    Variation
2017       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 4, 30m
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 4, 30m
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 4, 30m
2020       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 4, 30m
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 4, 30m
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 4, 30m
2023       4.7210           47  76.5957                2.7523             -8.5784     -46.3349       0.1019  40 , 4, 30m
2024      -0.46

/tmp/ipykernel_4453/1769088592.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Overall Combined
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio    Variation
2017       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 5, 30m
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 5, 30m
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 5, 30m
2020       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 5, 30m
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 5, 30m
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  40 , 5, 30m
2023      19.4792           43  81.3953                2.5754             -8.8323     -29.4252       0.6620  40 , 5, 30m
2024      -1.24

...

/tmp/ipykernel_4453/1769088592.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Overall Combined
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio    Variation
2017       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 2, 30m
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 2, 30m
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 2, 30m
2020       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 2, 30m
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 2, 30m
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 2, 30m
2023     -22.2078           30  76.6667                2.0572             -9.9320     -43.5658      -0.5098  50 , 2, 30m
2024      28.47

/tmp/ipykernel_4453/1769088592.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Overall Combined
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio    Variation
2017       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 3, 30m
2018       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 3, 30m
2019       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 3, 30m
2020       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 3, 30m
2021       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 3, 30m
2022       0.0000            0      NaN                   NaN                 NaN          NaN          NaN  50 , 3, 30m
2023     -30.5890           27  74.0741                2.0526            -10.2343     -45.9843      -0.6652  50 , 3, 30m
2024      18.80

/tmp/ipykernel_4453/1769088592.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


In [ ]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
# tb = await trade(weekly_data, i, j, k)
# tb

In [ ]:
sorted_stats

In [46]:
tb = await trade(weekly_data, 35, 5, "15m")
tb

,Index,Strike,Option Type,Expiry,Signal Generated At,Trade Type,Entry Date,Entry Time,Entry Price,Initial SL,Target,Exit Time,Exit Price,Points Captured,Slippages,After Costs,PnL,Remarks,Qty,ROI%,Trade Year,Trade Month
0,finnifty,19150,C,2023-01-10,2023-01-05 12:45:00,SELL,2023-01-05,13:00:00,30.1500,153.0000,10,2023-01-06 09:15:00,10.0000,20.1500,0.4015,19.7485,20735.9250,Target Hit,1050,2.0736,2023,1
1,finnifty,18500,C,2023-01-17,2023-01-12 13:00:00,SELL,2023-01-12,13:15:00,128.0500,230.5000,10,2023-01-16 09:15:00,230.5000,-102.4500,3.5855,-106.0355,-114518.3400,Initial SL Hit,1080,-11.4518,2023,1
2,finnifty,18500,P,2023-01-17,2023-01-13 10:00:00,SELL,2023-01-13,10:15:00,107.4000,176.7500,10,2023-01-16 09:15:00,10.0000,97.4000,1.1740,96.2260,103924.0800,Target Hit,1080,10.3924,2023,1
3,finnifty,18500,C,2023-01-17,2023-01-16 14:30:00,SELL,2023-01-16,14:45:00,56.6500,300.0500,10,2023-01-17 15:30:00,104.1500,-47.5000,1.6080,-49.1080,-53036.6400,Expiry Exit,1080,-5.3037,2023,1
4,finnifty,18600,P,2023-01-24,2023-01-19 12:45:00,SELL,2023-01-19,13:00:00,59.8500,99.6000,10,2023-01-23 09:15:00,10.0000,49.8500,0.6985,49.1515,53083.6200,Target Hit,1080,5.3084,2023,1
5,finnifty,18600,C,2023-01-24,2023-01-19 14:00:00,SELL,2023-01-19,14:15:00,207.7000,261.7500,10,2023-01-20 10:00:00,261.7500,-54.0500,4.6945,-58.7445,-63444.0600,Initial SL Hit,1080,-6.3444,2023,1
6,finnifty,18300,C,2023-02-07,2023-02-02 12:45:00,SELL,2023-02-02,13:00:00,60.5000,442.8500,10,2023-02-07 15:30:00,95.4000,-34.9000,1.5590,-36.4590,-39922.6050,Expiry Exit,1095,-3.9923,2023,2
7,finnifty,18300,P,2023-02-07,2023-02-03 11:00:00,SELL,2023-02-03,11:15:00,180.9500,662.8500,10,2023-02-07 10:30:00,10.0000,170.9500,1.9095,169.0405,185099.3475,Target Hit,1095,18.5099,2023,2
8,finnifty,18400,P,2023-02-14,2023-02-09 13:15:00,SELL,2023-02-09,13:30:00,88.5000,145.5500,10,2023-02-14 11:30:00,10.0000,78.5000,0.9850,77.5150,83716.2000,Target Hit,1080,8.3716,2023,2
9,finnifty,18600,P,2023-02-21,2023-02-16 12:45:00,SELL,2023-02-16,13:00:00,81.9000,167.6500,10,2023-02-17 12:15:00,167.6500,-85.7500,2.4955,-88.2455,-95305.1400,Initial SL Hit,1080,-9.5305,2023,2


In [47]:
tb_pe2 = tb[tb['Option Type'] == 'P']

In [48]:
stats2 = generate_stats(tb_pe2, 35, 5, "15m")
stats2

{7.643924924965621:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2017       0.0000            0      NaN                   NaN   
 2018       0.0000            0      NaN                   NaN   
 2019       0.0000            0      NaN                   NaN   
 2020       0.0000            0      NaN                   NaN   
 2021       0.0000            0      NaN                   NaN   
 2022       0.0000            0      NaN                   NaN   
 2023     134.7525           49  75.5102                6.6589   
 2024      34.4829           18  83.3333                4.9405   
 Overall  169.2354           67  77.6119                6.1632   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio    Variation  
 2017                    NaN          NaN          NaN  35 , 5, 15m  
 2018                    NaN          NaN          NaN  35 , 5, 15m  
 2019                    NaN          NaN          NaN  35 , 5, 15m  
 2020                    NaN          N

In [49]:
tb_pe2.to_csv('PSM finnifty pe_side.csv')

In [64]:
# def generate_stats(tb_expiry, signal_ma, no_of_candles, tf):
#     stats_df8 = pd.DataFrame(
#         index=range(2023, 2025),
#         columns=[
#             "Total ROI",
#             "Total Trades",
#             "Win Rate",
#             "Avg Profit% per Trade",
#             "Avg Loss% per Trade",
#             "Max Drawdown",
#             "ROI/DD Ratio",
#             "Variation",
#         ],
#     )
#     combined_df_sorted = tb_expiry
#     # combined_df_sorted = tb_expiry_ce
#     # combined_df_sorted = tb_expiry_pe
    
#     # Iterate over each year
#     for year in range(2023, 2025):
#         # Filter trades for the current year
#         year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
#         # Calculate total ROI
#         total_roi = year_trades["ROI%"].sum()
    
#         # Calculate total number of trades
#         total_trades = len(year_trades)
    
#         # Calculate win rate
#         win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
#         # Calculate average profit per trade
#         avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
#         # Calculate average loss per trade
#         avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
#         # Calculate maximum drawdown
#         max_drawdown = (
#             year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
#         ).min()
    
#         # Calculate ROI/DD ratio
#         roi_dd_ratio = total_roi / abs(max_drawdown)

#         variation = f'{signal_ma} , {no_of_candles}, {tf}'
    
#         # Store the statistics in the DataFrame
#         stats_df8.loc[year] = [
#             total_roi,
#             total_trades,
#             win_rate,
#             avg_profit,
#             avg_loss,
#             max_drawdown,
#             roi_dd_ratio,
#             variation,
#         ]
    
#     # Calculate overall statistics
#     overall_total_roi = stats_df8["Total ROI"].sum()
#     overall_total_trades = stats_df8["Total Trades"].sum()
#     overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
#     overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
#     overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
#     overall_max_drawdown = (
#         combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
#     ).min()
#     overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
#     overall_variation = variation
    
#     # Store the overall statistics in the DataFrame
#     stats_df8.loc["Overall"] = [
#         overall_total_roi,
#         overall_total_trades,
#         overall_win_rate,
#         overall_avg_profit,
#         overall_avg_loss,
#         overall_max_drawdown,
#         overall_roi_dd_ratio,
#         overall_variation,
#     ]
#     return {overall_roi_dd_ratio : stats_df8}

In [62]:
# stats = generate_stats(tb,)
# stats

# Base Variation of Positional Sell

In [98]:
# tb.to_csv('JJSU Positional Selling.csv')